<a href="https://colab.research.google.com/github/gabrielpelicolla/trabalhos_estudo_PLN/blob/main/Lista4PLN_GabrielPelicolla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação das Bibliotecas

##Instalar Hugging Face Transformers e Torch

In [11]:
!pip install transformers torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.4 MB/s eta 0:00:00


##Imports

In [12]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#Carregar modelo e tokenizer BERT (português - BERTimbau)

In [13]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

#Função para gerar embedding de frase

In [14]:
def get_bert_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()

#Exibir dependências de forma textual

In [8]:
for token in doc:
    print(f"{token.text:<12} {token.dep_:<10} {token.head.text:<10} {token.pos_:<10}")

O            det        cachorro   DET       
cachorro     nsubj      correu     NOUN      
correu       ROOT       correu     VERB      
atrás        advmod     correu     ADV       
do           case       gato       ADP       
gato         obl        atrás      NOUN      
que          nsubj      subiu      PRON      
subiu        acl:relcl  gato       VERB      
no           case       muro       ADP       
muro         obl        subiu      NOUN      
.            punct      correu     PUNCT     


#Frases de exemplo

In [16]:
frase1 = "O gato está em cima do telhado."
frase2 = "O felino subiu no telhado."
frase3 = "A economia brasileira cresceu este ano."
frase1, frase2, frase3

('O gato está em cima do telhado.',
 'O felino subiu no telhado.',
 'A economia brasileira cresceu este ano.')

#Gerar embeddings

In [17]:
embed1 = get_bert_embedding(frase1)
embed2 = get_bert_embedding(frase2)
embed3 = get_bert_embedding(frase3)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


#Calcular similaridades

In [18]:
sim12 = cosine_similarity(embed1, embed2)[0][0]
sim13 = cosine_similarity(embed1, embed3)[0][0]

In [19]:
print(f"Similaridade frase 1 x frase 2: {sim12:.4f}")
print(f"Similaridade frase 1 x frase 3: {sim13:.4f}")

Similaridade frase 1 x frase 2: 0.9095
Similaridade frase 1 x frase 3: 0.6551


 Interpretação esperada:
frase1 e frase2 são semanticamente parecidas (ambas sobre gatos e telhado).

frase1 e frase3 têm temas diferentes → baixa similaridade.